<a href="https://colab.research.google.com/github/PhuongAnhMannheim/-Team-Project-IPDW/blob/master/closed_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schema Mapping using closed feature extraction (FE)

##Set up the environment


In [0]:
# import required packages
import numpy as np
import pandas as pd
import re
import math

# access Google Drive via Google Cloud SDK
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import pandas as pd
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


## Loading the data sources

###dogfood title

In [2]:
## get identifier of source file
listed = drive.ListFile({'q': "title contains 'dogfood_all_names.csv'"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))
### id from AllContentJsonLaptop.json = 10mVUzmjWrrHX0OCVYrEpDlKryLuVzqjX
### id from dogfood_all_names.csv = 1_wYxyo-_rN6M5z6_1ZNIns3UrKCJ64gz

## download file
download_path = os.path.expanduser('~/data')
try:
  os.makedirs(download_path)
except FileExistsError:
  pass

title dogfood_all_names.csv, id 1_wYxyo-_rN6M5z6_1ZNIns3UrKCJ64gz


In [0]:
dogfood_title_file = os.path.join(download_path, 'dogfood_all_names.csv')
temp_file = drive.CreateFile({'id': '1_wYxyo-_rN6M5z6_1ZNIns3UrKCJ64gz'})
temp_file.GetContentFile(dogfood_title_file)

##loading the file into pandas
dogfood_title_data = pd.read_csv(dogfood_title_file)

In [4]:
dogfood_title_data.head()

,"name|""offerID"""
0,hills prescription diet canine j/d joint care|...
1,"royal canin giant starter?- mother babydog|""b..."
2,hills prescription diet canine d/d food sensit...
3,hills prescription diet canine z/d food sensit...
4,hills prescription diet canine u/d urinary car...


###laptop title

In [5]:
# get identifier of source file
listed = drive.ListFile({'q': "title contains 'laptop_all_names.csv'"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))
# id from AllContentJsonLaptop.json = 10mVUzmjWrrHX0OCVYrEpDlKryLuVzqjX
# id from dogfood_all_names.csv = 1_wYxyo-_rN6M5z6_1ZNIns3UrKCJ64gz
# id from laptop_all_names.csv = 1noquHOELRipJUPm7Vi0Hyvqz96xvaZ7U

## download file
download_path = os.path.expanduser('~/data')
try:
  os.makedirs(download_path)
except FileExistsError:
  pass

title laptop_all_names.csv, id 1noquHOELRipJUPm7Vi0Hyvqz96xvaZ7U


In [0]:
laptop_title_file = os.path.join(download_path, 'laptop_all_names.csv')
temp_file = drive.CreateFile({'id': '1noquHOELRipJUPm7Vi0Hyvqz96xvaZ7U'})
temp_file.GetContentFile(laptop_title_file)

##loading the file into pandas
laptop_title_data = pd.read_csv(laptop_title_file)

In [7]:
laptop_title_data.head()

,"name|""offerID"""
0,zeepad 8in 1gb 8gb qc zeepad x8 wht 0852170006...
1,lenovo 8 tab 4 8 16gb tablet wi fi slate black...
2,116 32gb flex 11 multi touch 2 in 1 chromebook...
3,lenovo za270025us 2 in 1 116ampquot touch scre...
4,lenovo?n23 yoga chromebook mediatek mtk 8173c ...


###dogfood description

###laptop description

## Building the dictionary


- Dictionary with multiple values for each key


*   For the prodecure on obtaining the dictionary content the filter options of each website have been observed
*   List item



In [0]:
dogfood_dict = {
    "brand": ['hills', 'hill\'s', 'Dr. Tim\'s'],
    "lifestage": ['Adult', 'Puppy', 'Senior', 'Large Breed', 'Small Breed', 'All Stages', 'All Lifestages', 'Junior', 'Not Specific', 'plus 7'], 
    "breed size": ['Extra Small', 'Giant', 'Large', 'Medium', 'Small', 'All Breeds', 'Not Specific', 'Maxi'],
    "breed type": ['Labrador', 'Pug', 'Beagle', 'Retriever', 'Golden Retriever', 'Bolognese', 'Chihuahua', 'Chinese Crested Dog', 'Jagd Terrier', 'Affenpinscher', 'Australian Terrier', 'Airedale Terrier', 'Alaskan Malamute', 'Alpine Dachsbracke', 'Aidi', 'Akita', 'Bernese Mountain Dog', 'Rottweiler', 'Saint Bernard', 'Mastiff', 'Newfoundland'],
    "food type": ['Dry', 'Canned', 'Freeze Dried', 'Frozen', 'Air-Dried', 'Digestive Aids', 'Pre-Mix', 'Hundefutter trocken', 'Hundefutter nass', 'Wet', 'Capsules', 'Dehydrated', 'Liquids', 'Sticks', 'Skin Care', 'Pain', 'Food Toppers', 'Food Roll', 'Food Pouch', 'Dehydrated Food', 'Freeze Dried Treats', 'Wet Food', ' Dry Food', ' Freeze-Dried', ' Food', ' Raw Food', ' Treats', ' Cold Treats'], 
    "flavour": ['Beef', 'Chicken', 'Egg', 'Lamb', 'Salmon', 'Turkey', 'Duck', 'Bison', 'Venison', 'Pork', 'Tuna', 'Trout', 'Whitefish', 'Herring', 'Rabbit', 'Liver', 'Bacon', 'Peanut Butter', 'Vegetable', 'Sweet Potato', 'Ham', 'Horse', 'Ostrich', 'Ox', 'Wild Boar'],
    "weight": ['4 lbs', '5 lbs', '7 lbs', '6 lbs', '30 lbs', '40 lbs']    
}


In [54]:
print(dogfood_dict['brand'][0])

hills


In [55]:
print(dogfood_dict['brand'])

['hills', "hill's", "Dr. Tim's"]


In [56]:
len(dogfood_dict)

7

In [57]:
len(dogfood_dict['weight'])

6

In [58]:
def foo(str_attr):
  if str_attr in dogfood_dict['brand']:
    return str_attr
  
foo('hill\'s')

"hill's"

In [48]:
def foo_loop(str_attr):
  i = 1
  while i <= len(dogfood_dict):
    if str_attr in dogfood_dict[i]:
      return str_attr
  
foo_loop('hills')

KeyError: ignored

In [0]:
laptop_dict = {
    "condition": ['Refurbished Laptops', ' All Laptop Computers', ' New', ' Pre-Owned', ' Refurbished', ' Open-Box'],
    "type": ['2-in-1 Laptops', ' Touchscreen Laptops', ' Gaming Laptops', ' Google Chromebooks', ' Windows 10 Laptops', ' Ultrabook Laptops', ' Apple Laptops', ' Laptops', ' Windows 10 PCs', ' MacBooks', ' Notebooks', ' Ultrabook', ' Workstations', ' Notebook', ' Mobile Workstation', ' Gaming Notebook', ' 2 in 1 Chromebook', ' Netbook', ' Mobile Workstation Ultrabook', ' 2 in 1 Ultrabook', ' 2 in 1 Mobile Workstation', ' Thin Client Notebook'],
    "brand": ['HP Laptops', 'Dell Laptops', 'Lenovo Laptops', 'ASUS Laptops', 'Samsung Laptops', 'Alienware Laptops', 'HP Omen Laptops', 'Acer Laptops', 'Toshiba Laptops', 'Surface Laptops', 'MSI Laptops', 'Microsoft', 'Dell', 'HP', 'Lenovo', 'ASUS', 'Acer', 'Samsung', 'MSI', 'Alienware', 'LG', 'Razer', 'Toshiba', 'Hyundai', 'Wacom', 'HP OMEN', 'Asus', 'Dell Computer', 'Google', 'HUAWEI', 'Lenovo', 'LG', 'Panasonic', 'Samsung', 'Toshiba  Acer', 'Apple', 'Ematic', 'EVGA', 'Sony', 'Fijitsu', 'HP Inc', 'CTL'],
    "OS": ['Google Chrome', 'HP ThinPro', 'Windows', 'Windows 7', 'Windows 7 Professional', 'Windows 8', 'Windows 8.1', 'Windows 8.1 Professional', 'Windows 10', 'Windows 10 Professional', 'macOS', 'Mac OS X', 'Windows 10 Pro', 'Windows S', 'Windows 10', 'Windows 7 Pro/ Windows 10 Pro', 'Windows 7 Pro 64 bit', 'Windows 7 Pro'],
    "RAM size": ['1GB', '2GB', '4GB', '6GB', '8GB', '12GB', '16GB', '32GB', '64GB', '32 gigabytes', '16 gigabytes', '12 gigabytes', '8GB', '8 gigabytes', '6 gigabytes', '4 gigabytes', '2 gigabytes'],
    "screen size": ['10 Inch Laptops', '12 Inch Laptops', '13 Inch Laptops', '14 Inch Laptops', '15 Inch Laptops', '10"', '10.1"', '12"', '12.2"', '12.3"', '12.5"', '13"', '13.5"', '14"', '15"', '11.6"', '13.1"', '13.3"', '13.9"', '15.4"', '15.6"', '17"', '17.3"', '18.4"'],
    "graphic card brand": ['AMD', 'Intel', 'None', 'NVIDIA', 'Other'],
    "hard drive size": ['1TB', '1TB + 1TB', '1TB + 128GB', '1TB + 256GB', '2TB', '4TB', '16GB', '32GB', '64GB', '128GB', '256GB', '500GB', '512GB'],
    "processor model": ['Intel 1st Generation Core i5', 'Intel 2nd Generation Core i5', 'Intel 2nd Generation Core i7', 'Intel 3rd Generation Core i5', 'Intel 3rd Generation Core i7', 'Intel 4th Generation Core i3', 'Intel 4th Generation Core i5', 'Intel 4th Generation Core i7', 'Intel 5th Generation Core i3', 'Intel 5th Generation Core i5', 'Intel 5th Generation Core i7', 'Intel 6th Generation Core i3', 'Intel 6th Generation Core i5', 'Intel 6th Generation Core i7', 'Intel 7th Generation Core i3', 'Intel 7th Generation Core i5', 'Intel 7th Generation Core i7', 'Intel 7th Generation Core i9', 'Intel 8th Generation Core i3', 'Intel 8th Generation Core i5', 'Intel 8th Generation Core i7', 'Intel 8th Generation Core i9', 'Intel Atom', 'Intel Celeron', 'Intel Core2 Duo', 'Intel Core i5', 'Intel Core i7', 'Intel Core M', 'Intel Core m3', 'Intel Core m5', 'Intel Core m7', 'Intel Pentium'],
    "processor brand": ['Intel', 'AMD', 'Samsung', 'MediaTek', 'ARM']
}

##Building extraction functionalities

###overlap

In [0]:
class Dogfood_Title_Overlap():
    def __init__(self,list_content,list_id,list_attr,attr_select):
        self.list_content=list_content
        self.list_id=list_id
        self.list_attr=list_attr
        self.attr_select=attr_select
    
    def get_attribute(self):
        dict_item = dict()
        for i in range(len(self.list_id)):
            list_row = list()
            str_single = self.list_content[i]
            for pos in range(len(self.list_attr)):
                attr = eval("self.%s(str_single)" % ("get_" + list_attr[pos]))
                list_row.append(attr)
            dict_item[self.list_id[i]] = list_row

        return dict_item
  
    def get_brand(self,str_single):
        bow = str_single.split(' ')
        intersection(bow,dogfood_dict['brand'])
      
    def intersection(lst1, lst2):
      
        return list(set(lst1) & set(lst2))
    

In [68]:
#try-out of overlap function
if __name__=="__main__":
    list_attr=[
        "brand"
    ]
    str1 = "hills science diet adult 7+ perfect weight chicken recipe dry dog food 40oz x12 bag"
    str2 = "hills science diet adult 11+ small  toy breed age defying chicken meal rice  barley recipe dry dog food case of 20 4.5 kg bag"
    list1 = [str1, str2]
    list2 = ["str1","str2"]
    rd=Dogfood_Title_Overlap(list1,list2,list_attr,dogfood_dict)
    dict_item=rd.get_attribute()
    print(dict_item)

{'str1': [None], 'str2': [None]}


In [60]:
str1 = "hills science diet adult 7+ perfect weight chicken recipe dry dog food 40oz x12 bag"

str1 = str1.split(' ')

str1

['hills',
 'science',
 'diet',
 'adult',
 '7+',
 'perfect',
 'weight',
 'chicken',
 'recipe',
 'dry',
 'dog',
 'food',
 '40oz',
 'x12',
 'bag']

In [61]:
dogfood_dict['brand']

['hills', "hill's", "Dr. Tim's"]

In [63]:
str1 = "hills science diet adult 7+ perfect weight chicken recipe dry dog food 40oz x12 bag"
str1 = str1.split(' ')

def intersection(lst1, lst2):
  return list(set(lst1) & set(lst2))

intersection(str1, dogfood_dict['brand'])



['hills']

## Text feature extraction

retrieve feature distribution
calculate as % (from the total of webpages?)


###save / store extracted value in csv